In [41]:
# --- Now, a concrete implementation using ASE ---
from ase import Atoms
from ase.calculators.lj import LennardJones
calc = LennardJones(sigma=1, epsilon=1, rc=3, smooth=False)
atoms = Atoms('X2')
atoms.calc = calc

print(atoms.get_potential_energy())

nan


In [ ]:
from abc import ABC, abstractmethod
import numpy as np

###############################
# Base Potential Class (Abstract)
###############################

class PotentialEnergySurface(ABC):
    """Abstract base class for potential energy surfaces."""

    def __init__(self): 
        self.max_acceptable_force_mag = np.inf # will be updated by ABC later
        self.energy_calls = 0 
        self.force_calls = 0
    
    @abstractmethod
    def _potential(self, position: np.ndarray) -> float: 
        """Compute potential energy at given position."""
        pass

    def potential(self, position: np.ndarray) -> float: 
        """
        Wrapper for potential calls. 
        
        DO NOT EDIT: Users should implement _potential()
        """
        self.energy_calls += 1
        return self._potential(position)
        
    @abstractmethod
    def default_starting_position(self) -> np.ndarray:
        """Return default starting position for this PES."""
        pass
    
    def _gradient(self, position) -> np.ndarray:
        """Compute analytic gradient at given position, if available
        Raise NotImplementedError if not implemented.
        
        If your potential has no analytical gradient, simply omit this method 
        from your implementation, and the ABC will perform finite-difference.
        """
        raise NotImplementedError("Analytic gradient not implemented for this PES.")
    
    def gradient(self, position) -> np.ndarray:
        """
        Wrapper for _gradient with built-in force magnitude limiting
        DO NOT EDIT: Users should implement _gradient() for analytical gradient calculation
        """
        if (norm := np.linalg.norm(grad := self._gradient(position))) > self.max_acceptable_force_mag: 
            print(f"Warning: Gradient value of {grad} detected as likely unphysically large in magnitude; shrunk to magnitude {self.max_acceptable_force_mag}")
            grad = self.max_acceptable_force_mag * grad / norm
        
        self.force_calls += 1
        return grad       

    def plot_range(self) -> tuple:
        """Return plotting range for visualization."""
        return None
        
    def known_minima(self) -> list[np.ndarray]:
        """Return known basins (for analysis)."""
        return None 

    def known_saddles(self) -> list[np.ndarray]:
        """Return known saddles (for analysis)."""
        return None


###############################
# Concrete Potential Implementations
###############################

class DoubleWell1D(PotentialEnergySurface):
    """1D double well potential."""
    
    def _potential(self, x):
        """Compute double well potential with minima at x=-1 and x=1."""
        return 1/6 * (5 * (x**2 - 1))**2
    
    def _gradient(self, x):
        return np.array([50/3 * x * (x**2-1)])
        
    def default_starting_position(self):
        return np.array([-1.0], dtype=float)
        
    def plot_range(self):
        return (-2, 2)
        
    def known_minima(self):
        return [np.array([-1.0], dtype=float), np.array([1.0], dtype=float)]
    
    def known_saddles(self):
        return [np.array([0.0], dtype=float)]

class Complex1D(PotentialEnergySurface):
    
    def _potential(self, x):
        a=[6.5, 4.2, -7.3, -125]
        b=[2.5, 4.3, 1.5, 0.036]
        c=[9.7, 1.9, -2.5, 12]
        V = x**2
        for i in range(4):
            exponent = -b[i]*(x-c[i])**2
            exponent = np.clip(exponent, -100, 100)
            V += a[i]*np.exp(exponent)
        return V
     
    def default_starting_position(self):
        return np.array([0.0], dtype=float)
        
    def plot_range(self):
        return (-3.5, 11.6)
        
    def known_minima(self):
        return [
                np.array([-2.27151]), 
                np.array([0.41295]), 
                np.array([2.71638]), 
                np.array([8.69999]), 
                np.array([10.35518]) 
                ]
    
    def known_saddles(self):
        return [
                np.array([-1.2645]),
                np.array([1.94219]), 
                np.array([4.55508]),
                np.array([9.7913])
                ]


import numpy as np

class StandardMullerBrown2D(PotentialEnergySurface):
    """2D Muller-Brown potential."""

    def __init__(self):
        super().__init__()
        self.A = np.array([-200, -100, -170, 15])
        self.a = np.array([-1, -1, -6.5, 0.7])
        self.b = np.array([0, 0, 11, 0.6])
        self.c = np.array([-10, -10, -6.5, 0.7])
        self.x0 = np.array([1, 0, -0.5, -1])
        self.y0 = np.array([0, 0.5, 1.5, 1])

    def _potential(self, pos):
        """Compute the Muller-Brown potential with numerical safeguards."""
        x, y = pos[0], pos[1]

        V = 0.0
        for i in range(4):
            dx = x - self.x0[i]
            dy = y - self.y0[i]
            exponent = self.a[i]*dx**2 + self.b[i]*dx*dy + self.c[i]*dy**2
            exponent = np.clip(exponent, -100, 100)
            V += self.A[i] * np.exp(exponent)
        return V

    def _gradient(self, position):
        x, y = position[0], position[1]

        dVdx = 0.0
        dVdy = 0.0

        for i in range(4):
            dx = x - self.x0[i]
            dy = y - self.y0[i]
            exponent = self.a[i]*dx**2 + self.b[i]*dx*dy + self.c[i]*dy**2
            exp_term = np.exp(np.clip(exponent, -100, 100))
            dVdx += self.A[i] * exp_term * (2*self.a[i]*dx + self.b[i]*dy)
            dVdy += self.A[i] * exp_term * (self.b[i]*dx + 2*self.c[i]*dy)
        
        grad = np.array([dVdx, dVdy])            
        return grad 

    def default_starting_position(self):
        return np.array([0.0, 0.0], dtype=float)

    def plot_range(self):
        return ((-2, 2), (-1, 2))

    def known_minima(self):
        return [
            np.array([-0.5582236346, 1.441725842]),  # Basin A
            np.array([0.6234994049, 0.02803775853]), # Basin B  
            np.array([-0.050010823, 0.4666941049])   # Basin C
        ]

    def known_saddles(self):
        return [
            np.array([0.212486582, 0.2929883251]),   # Transition A<-->B
            np.array([-0.8220015587, 0.6243128028])  # Transition B<-->C
        ]
    

# --- Now, a concrete implementation using ASE ---
from ase import Atoms
from ase.calculators.lj import LennardJones

class ASEPotentialEnergySurface(PotentialEnergySurface):
    """
    A base class for PES implementations that use ASE calculators.
    """
    def __init__(self, ase_atoms, calculator):
        super().__init__()
        self.atoms = ase_atoms
        if calculator is not None: 
            self.atoms.calc = calculator

    def _potential(self, position):
        """Compute potential energy at given position using ASE."""
        # Ensure 'position' is a numpy array of correct shape for ASE
        # For N atoms, it should be (N, 3)
        self.atoms.positions = position.reshape(-1, 3)
        return self.atoms.get_potential_energy()

    def _gradient(self, position):
        """Compute gradient at given position using ASE."""
        self.atoms.positions = position.reshape(-1, 3)
        # ASE returns forces, which are negative gradients
        forces = self.atoms.get_forces()
        return -forces.flatten() # Flatten to match your 'position' input shape


from ase import Atoms
from ase.calculators.lj import LennardJones
import numpy as np

class LennardJonesCluster(ASEPotentialEnergySurface):
    def __init__(self, num_atoms, initial_positions=None, 
                 sigma=1.0, epsilon=1.0, min_distance=0.9, padding=0.5):
        """
        A smarter LJ cluster implementation that:
        - Automatically scales the box size with num_atoms^(1/3)
        - Ensures particles aren't too close (avoids huge repulsive forces)
        - Can generate reasonable random or lattice initial positions
        
        Args:
            num_atoms: Number of LJ particles
            initial_positions: Optional starting positions (None for auto-generated)
            sigma: LJ σ parameter
            epsilon: LJ ε parameter
            min_distance: Minimum allowed distance between particles (in units of σ)
            padding: Extra space around cluster (in units of σ)
        """
        self.num_atoms = num_atoms
        self.sigma = sigma
        self.epsilon = epsilon
        self.min_distance = min_distance * sigma
        self.padding = padding * sigma

        # Calculate reasonable box size
        self.box_size = self._calculate_box_size(num_atoms)
        
        # Generate initial positions if not provided
        if initial_positions is None:
            initial_positions = self.default_starting_position()
        
        # Create ASE Atoms object
        atoms = Atoms(symbols=['X']*num_atoms, 
                     positions=initial_positions.reshape(-1,3),
                     pbc=False)
        
        # Setup LJ calculator
        calculator = LennardJones(sigma=sigma, epsilon=epsilon, 
                                rc=3*sigma, smooth=False)
        
        atoms.calc = calculator

        super().__init__(atoms, None)

    def _calculate_box_size(self, num_atoms):
        """Calculate box size based on particle count and LJ parameters"""
        # Volume scales with number of particles
        volume_per_atom = (4/3) * np.pi * (self.min_distance/2)**3
        total_volume = num_atoms * volume_per_atom
        
        # Cubic root to get linear dimension
        linear_size = total_volume ** (1/3)
        
        # Add padding and convert to box length
        return linear_size + 2*self.padding

    def default_starting_position(self):
        """Generate initial positions that avoid extreme forces"""
        positions = np.zeros((self.num_atoms, 3))
        
        # First particle at origin
        positions[0] = [0, 0, 0]
        
        # Place subsequent particles with reasonable spacing
        for i in range(1, self.num_atoms):
            while True:
                # Random position in sphere of decreasing radius
                r = self.min_distance * (1 + 0.5*i)
                pos = np.random.uniform(-r, r, size=3)
                
                # Check distances to existing particles
                valid = True
                for j in range(i):
                    if np.linalg.norm(pos - positions[j]) < self.min_distance:
                        valid = False
                        break
                
                if valid:
                    positions[i] = pos
                    break
        
        # Center the cluster
        positions -= np.mean(positions, axis=0)
        
        return positions.flatten()

    def known_minima(self):
        """Return known minima for small clusters"""
        if self.num_atoms == 2:
            return [np.array([0,0,0, 0,0,1.12*self.sigma])]  # Diatomic minimum
        elif self.num_atoms == 3:
            return [np.array([0,0,0, 0,0.5*1.12*self.sigma,0.866*1.12*self.sigma, 
                            0,-0.5*1.12*self.sigma,0.866*1.12*self.sigma])]  # Equilateral triangle
        # Add more known minima as needed
        return []

    def known_saddles(self):
        """Return known transition states for small clusters"""
        return []

In [43]:
mypot = LennardJonesCluster(2)
mypot.gradient(mypot.default_starting_position())

array([-0.97071992,  1.01644186, -0.09019999,  0.97071992, -1.01644186,
        0.09019999])

In [46]:
from numpy import array

ms = [array([ 0.45580241,  0.29975277,  0.52179542, -0.21605851, -0.59245761,
        0.63352984]), array([ 0.36166507,  0.08112468,  0.6680567 ,  0.33412726, -0.95378262,
        0.23431761]), array([ 0.65501316,  0.21041675,  0.64708384,  0.44878615, -0.83184561,
        0.28503325]), array([ 0.6648861 ,  0.25209209,  0.5170549 ,  0.35601014, -0.82702809,
        0.52109335]), array([ 0.66592189,  0.25642632,  0.52230116,  0.35517685, -0.82214786,
        0.51627673])]

print([np.linalg.norm(arr) for arr in ms])

[np.float64(1.170010845239571), np.float64(1.2883825695124027), np.float64(1.3662558418380324), np.float64(1.3620670608578265), np.float64(1.3603765226308688)]


In [1]:
from abc import ABC, abstractmethod
import numpy as np
from copy import deepcopy

class Optimizer(ABC):
    """Abstract base class for all optimizer backends"""
    
    def __init__(self, abc_sim):
        self.abc_sim = abc_sim
        self._reset_state()

    def _reset_state(self):
        """Initialize/clear all trajectory tracking"""
        
        self.trajectory = []
        self.unbiased_energies = []
        self.biased_energies = []
        self.unbiased_forces = []
        self.biased_forces = []
        self._last_x = None

    def _compute(self, x):
        """Compute and record energy/forces (cached)"""
        if self._last_x is None or not np.allclose(x, self._last_x):
            self._last_x = x.copy()
            
            # Your existing computation logic
            unbiased_energy = self.abc_sim.potential.potential(x)
            biased_energy = self.abc_sim.compute_biased_potential(x, deepcopy(unbiased_energy)) # reuse unbiased call from before

            try:
                unbiased_force = - self.abc_sim.potential.gradient(x)
            except NotImplementedError:
                unbiased_force = None
            biased_force = self.abc_sim.compute_biased_force(x, unbiased_force)
           
            # Record
            self.trajectory.append(x.copy())
            self.unbiased_energies.append(unbiased_energy)
            self.biased_energies.append(biased_energy)
            self.unbiased_forces.append(unbiased_force)
            self.biased_forces.append(biased_force)
        
        return self.biased_energies[-1], -self.biased_forces[-1]  # (energy, -gradient)


    def get_traj_data(self):
        accepted = self._accepted_steps()
        indices = []
        for pos in accepted:
            found = False
            for i, traj_pos in enumerate(self.trajectory):
                if np.allclose(pos, traj_pos, atol=1e-10):
                    indices.append(i)
                    found = True
                    break
            if not found:
                # Try with a higher tolerance
                for i, traj_pos in enumerate(self.trajectory):
                    if np.allclose(pos, traj_pos, atol=1e-8):
                        indices.append(i)
                        found = True
                        print(f"Warning: Position {pos} not found with tol={1e-10}, matched with tol={1e-8} at index {i}.")
                        break
            if not found:
                print(f"Warning: Accepted position {pos} not found in trajectory with any tolerance.")
        traj = [self.trajectory[i] for i in indices]
        unbiased_e = [self.unbiased_energies[i] for i in indices]
        biased_e = [self.biased_energies[i] for i in indices]
        unbiased_f = [self.unbiased_forces[i] for i in indices]
        biased_f = [self.biased_forces[i] for i in indices]
        return (traj, unbiased_e, biased_e, unbiased_f, biased_f)

    def descend(self, x0, max_steps=None, convergence_threshold=None):
        """Universal descent method (same for all backends)"""
        self._reset_state()
        result = self._run_optimization(x0, max_steps, convergence_threshold)
        return result, self.get_traj_data()
    
    @abstractmethod
    def _run_optimization(self, x0, max_steps, convergence_threshold):
        """
        Backend-specific optimization (implemented by child classes)
        Should call _compute()
        """
        pass

    @abstractmethod
    def _accepted_steps():
        """
        Return a list of the positions that were actually accepted by the optimizer, in the order they were accepted
        
        Would be nicer if could get the indices, but many optimizer types (e.g. SciPy) do not allow for this
        """
        pass 


from scipy.optimize import minimize
from scipy.sparse.linalg import LinearOperator
import numpy as np


class ScipyOptimizer(Optimizer):
    def __init__(self, abc_sim, method='BFGS', **kwargs):
        """
        SciPy optimizer supporting BFGS, L-BFGS-B, and CG methods.
        
        Args:
            method: One of 'BFGS', 'L-BFGS-B', or 'CG'
            kwargs: Method-specific options
        """
        super().__init__(abc_sim)
        if method not in ['BFGS', 'L-BFGS-B', 'CG']:
            raise ValueError(f"Method '{method}' not supported. Choose from: BFGS, L-BFGS-B, CG")
        self.method = method
        self.optimizer_kwargs = kwargs
        self._result = None
        self.accepted_steps = []

    def _run_optimization(self, x0, max_steps=None, convergence_threshold=None):
        self._reset_state()
        self.accepted_steps = [x0.copy()]
        last_accepted_x = x0.copy()
        
        options = self.optimizer_kwargs.copy()
        if max_steps is not None:
            options['maxiter'] = max_steps
            # Some methods use different names for max iterations
            options['maxfun'] = max_steps * 5  # For methods that count function evaluations
            options['maxls'] = max_steps  # For line search steps

        def callback(x):
            nonlocal last_accepted_x
            if not np.allclose(x, last_accepted_x):
                self._compute(x)  # Ensure cached
                self.accepted_steps.append(x.copy())
                last_accepted_x = x.copy()

        self._result = minimize(
            fun=lambda x: self._compute(x)[0],
            x0=x0,
            method=self.method,
            jac=lambda x: self._compute(x)[1],
            tol=convergence_threshold,
            callback=callback,
            options=options
        )

        return self._package_result()
    
    def _construct_l_bfgs_hess_inv(self, hess_inv_operator: LinearOperator):
        """Convert L-BFGS-B LinearOperator inverse Hessian to dense matrix"""
        n = self.abc_sim.dimension
        I = np.eye(n)
        
        # Apply the LinearOperator to each basis vector
        inv_H = np.column_stack([hess_inv_operator.matvec(I[:, i]) for i in range(n)])
        
        # Symmetrize the result
        inv_H = 0.5 * (inv_H + inv_H.T)
        
        return inv_H

    def _package_result(self):
        """Standardized result format"""
        result = {
            'x': self._result.x,
            'converged': self._result.success,
            'nit': self._result.nit,
            'message': self._result.message
        }
        
        # Handle Hessian information differently for each method
        if hasattr(self._result, 'hess_inv'):
            if self.method == 'BFGS':
                # BFGS provides dense matrix directly
                result['hess_inv'] = self._result.hess_inv
            elif self.method == 'L-BFGS-B':
                # L-BFGS-B needs conversion from LinearOperator
                result['hess_inv'] = self._construct_l_bfgs_hess_inv(self._result.hess_inv)
        
        return result

    def _accepted_steps(self):
        return self.accepted_steps.copy()

    @property 
    def inv_hessian(self):
        """Get inverse Hessian approximation if available"""
        if not hasattr(self, '_result'):
            raise AttributeError("Optimization not yet run")
        
        if self.method == 'BFGS' and hasattr(self._result, 'hess_inv'):
            return self._result.hess_inv
        elif self.method == 'L-BFGS-B' and hasattr(self._result, 'hess_inv'):
            return self._construct_l_bfgs_hess_inv(self._result.hess_inv)
        
        raise AttributeError(f"Inverse Hessian not available for method '{self.method}'")

In [51]:
from ca_abc import CurvatureAdaptiveABC
from potentials import *
mypot = Complex1D()
myabc = CurvatureAdaptiveABC(mypot)

myopt = ScipyOptimizer(myabc)


result = myopt.descend([18], 2, 1e-5)
result

/tmp/ipykernel_60159/2599499563.py:139: OptimizeWarning: Unknown solver options: maxfun, maxls
  self._result = minimize(


({'x': array([2.55226954]),
  'converged': False,
  'nit': 2,
  'message': 'Maximum number of iterations has been exceeded.',
  'hess_inv': array([[0.28664761]])},
 ([array([18.]), array([12.95]), array([2.55226954])],
  [array([289.79698708]), array([46.69848374]), array([2.1604557])],
  [array([289.79698708]), array([46.69848374]), array([2.1604557])],
  [None, None, None],
  [array([-50.77570158]), array([-34.17667471]), array([2.09688938])]))